In [36]:
import pandas as pd
import numpy as np
import math
import itertools
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1.inset_locator import (inset_axes, InsetPosition, mark_inset)
import math
import copy
import re
from collections import OrderedDict 
import Bio
from Bio import SeqIO
import xls_analysis_classes
from xls_analysis_classes import *

#from xls_analysis_classes import *

matplotlib.rcParams.update({'font.size': 10})

In [40]:
d = pd.read_csv('./proteasome-tri-crosslinks.csv', sep = ',')
data = pd.DataFrame(d)
#data.insert(2, 'dummy A', 'prot1')
#data.insert(3, 'dummy XLA', 'K1')
data.insert(2, 'Subunit DB', 'prot1')
data.insert(3, 'XL DB', 'K1')
#
data.insert(6, 'dummy B', 'prot1')
data.insert(7, 'dummy XLB', 'K1')
data.insert(10, 'dummy C', 'prot1')
data.insert(11, 'dummy XLC', 'K1')
#print(data)
df1 = data.iloc[:, :4].join(data.iloc[:, 12:13]).join(data.iloc[:, 15:16]).join(data.iloc[:, 18:32]).join(data.iloc[:, 46:53])
df1.insert(6, 'Common/Gene Name B', 'prot1')
print(df1.columns)
df2 = data.iloc[:, 4:8].join(data.iloc[:, 13:14]).join(data.iloc[:, 16:17]).join(data.iloc[:, 18:25]).join(data.iloc[:, 32:39]).join(data.iloc[:, 46:53])
#print(df2.columns)
df3 = data.iloc[:, 8:12].join(data.iloc[:, 14:15]).join(data.iloc[:, 17:18]).join(data.iloc[:, 18:25]).join(data.iloc[:, 39:46]).join(data.iloc[:, 46:53])
#print(df3.columns)
df1.to_csv('proteasome-tri-crosslinks-1.csv', index = False)
df2.to_csv('proteasome-tri-crosslinks-2.csv', index = False)
df3.to_csv('proteasome-tri-crosslinks-3.csv', index = False)
#print(df3)

Index(['Subunit A', 'XL A', 'Subunit DB', 'XL DB', 'Subunit Name A',
       'Common/Gene Name A', 'Common/Gene Name B', 'XL Scan', 'XL RT',
       'XL m/z', 'XL Charge', 'XL PPM', 'Corr. XL PPM', 'Corr. XL Isotope',
       'm/z A', 'Charge A', 'Sequence A', 'Protein Mods A', 'PPM A', 'Score A',
       'EV A', 'Ambiguous', 'Redundant', 'Subunit Names', 'Linkage Info',
       'Redundant Count', 'Unique Count', 'CLMS-CSV'],
      dtype='object')


In [70]:
# open a file which is down two directories and called "definitions.txt" and read it
# open this file as a pandas dataframe and name the columns as genename, genericname
heads = ['genename', 'genericname', 'chain1', 'chain2', 'subunit name']
dnames = pd.read_csv('../../definitions.txt', names = heads)
#-------------------------------------------------------------------------------------
# Define a function that replaces or inserts the chain1 based on the subunit name
#-------------------------------------------------------------------------------------
def set_chain_id1(sname):
    # Check if the subunit name is NaN or empty
    if pd.isna(sname) or sname == '':
        return None
    # Find indices where the subunit name matches the sname
    indices = dnames[dnames['subunit name'].str.contains(sname, na=False)].index.tolist()
    # If there is a match, return the corresponding chain1 value
    if indices:
        return str(dnames.loc[indices[0], 'chain1'])
    return None

d['Chain ID A'] = d['Subunit A'].apply(set_chain_id1)
d['Chain ID B'] = d['Subunit B'].apply(set_chain_id1)
d['Chain ID C'] = d['Subunit C'].apply(set_chain_id1)
print(d['Chain ID B'].dropna())
columns_to_new = ['Chain ID A', 'XL A', 'Chain ID B', 'XL B']
xl_AB = d[columns_to_new].dropna()
unique_xl_AB = xl_AB.drop_duplicates()
# Split the 'XL A' and 'XL B' columns by ';' and expand into separate rows
unique_xl_AB = unique_xl_AB.assign(**{
    'XL A': unique_xl_AB['XL A'].str.split(r'[;|]'),
    'XL B': unique_xl_AB['XL B'].str.split(r'[;|]')
}).explode('XL A').explode('XL B')
unique_xl_AB['XL A'] = unique_xl_AB['XL A'].str.replace('K', '', regex=False)
unique_xl_AB['XL B'] = unique_xl_AB['XL B'].str.replace('K', '', regex=False)
# write the dataframes to csv files
unique_xl_AB.to_csv('unique_xl_AB.csv', index=False)
xl_AB.to_csv('xl_AB.csv', index=False)
columns_to_new = ['Chain ID A', 'XL A', 'Chain ID C', 'XL C']
xl_AC = d[columns_to_new].dropna()
xl_AC.to_csv('xl_AC.csv', index=False)
columns_to_new = ['Chain ID B', 'XL B', 'Chain ID C', 'XL C']
xl_BC = d[columns_to_new].dropna()
xl_BC.to_csv('xl_BC.csv', index=False)
#------------------------------------------------------------------------------------- 

0        1
1        5
2        5
3        5
4        5
        ..
3433     E
3434     E
3435     E
3436     E
3437     E
Name: Chain ID B, Length: 3096, dtype: object
